## IMPORT LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import json
import re
import shlex, subprocess, os
import time
import csv
import glob
from subprocess import Popen, PIPE
from randomwordgenerator import randomwordgenerator
import random

## READ FILES

In [3]:
docs = json.load(open('document_passages.json'))
example_passage = docs['543']['42']

stop = json.load(open('SMARTstop.json'))

In [12]:
finput = open('input_file.txt', 'w+')

for i in range(len(docs)):
    for doc_id in docs[str(i)].keys():
        finput.write('<DOC>\n')
    
        finput.write('<DOCNO>'+ str(i)+'-'+str(doc_id) +'</DOCNO>\n')
        finput.write('<TEXT>\n')
        finput.write(docs[str(i)][str(doc_id)]+'\n')
        finput.write('</TEXT>\n')
        finput.write('</DOC>\n')
    
finput.close()

In [6]:
dev = pd.read_csv('dev.tsv', sep ='\t')
train = pd.read_csv('train.tsv', sep ='\t')
test = pd.read_csv('test.tsv', sep ='\t')

In [4]:
train.loc[train['DocumentID'] == 0]

,QID,Question,DocumentID,DocumentName,RelevantPassages
119,1758,Then What Are The Main Focuses Of Economics?,0,Economics.html,"0,1"
1424,1760,How Do We Define Microeconomics?,0,Economics.html,6
2815,1761,"How Does The Idea Of ""Free Markets"" Relate To ...",0,Economics.html,6
2960,1757,"As A Discipline, Is Economics Really A Legitim...",0,Economics.html,0


## CREATE QREL FILES FOR TREC

In [44]:
f = open('qrel_train.txt', 'w+')
for i in range(len(train)):
    for item in train['RelevantPassages'][i].split(','):
        f.write(str(train['QID'][i])+' '+'Q0'+' '+ str(train['QID'][i])+'-'+str(item)+ ' 1\n')
f.close()

In [41]:
f = open('qrel_train.txt', 'w+')
for i in range(len(train)):
    for item in train['RelevantPassages'][i].split(','):
        f.write(str(train['QID'][i])+' '+'Q0'+' '+ str(train['DocumentID'][i])+'-'+str(item)+ ' 1\n')
f.close()

In [35]:
fd = open('qrel_dev.txt', 'w+')
for i in range(len(dev)):
    for item in dev['RelevantPassages'][i].split(','):
        fd.write(str(dev['QID'][i])+' '+'Q0'+' '+ str(dev['DocumentID'][i])+'-'+str(item)+ ' 1\n')
fd.close()

# ft = open('qrel_test.txt', 'w+')
# for i in range(len(test)):
#     for item in test['RelevantPassages'][i].split(','):
#         ft.write(str(test['QID'][i])+' '+'Q0'+' '+ str(test['DocumentID'][i])+'-'+str(item)+ ' 1\n')
# ft.close()

In [17]:
def prepare_question_text(q_id, question_text):
    question_text = question_text.replace('?','') #remove ?
    question_text = question_text.replace("'",'')
    question_text = question_text.replace('"','')
    question_text = question_text.replace('-',' ')
    question_text = question_text.replace('(','')
    question_text = question_text.replace(')','')
    question_text = question_text.replace(',','')
    question_text = question_text.replace('.','')
    question_text = question_text.replace('&',' and ')
    question_text = question_text.replace(':','')
    question_text = question_text.replace('>','')#error in dataset

    if "[" in question_text:
        #print q_id,question_text
        if q_id == "3340": #remove contents
            question_text = re.sub(r'\[[^\(]*?\]', r'', question_text)
        else: #keep contents
            question_text = re.sub(r'\[(?:[^\]|]*\|)?([^\]|]*)\]', r'\1', question_text)
    if "/" in question_text:
        #print q_id,question_text
        if q_id == "104" or q_id == "857":
            question_text = question_text.replace('/','')
        else:
            question_text = question_text.replace('/',' or ')
    
    lower = question_text.lower()
    stopW = [w for w in lower.split() if not w in stop]
    return ' '.join(stopW)
#     return question_text.lower()

## CREATE QUERY FILES FOR INDRI

In [74]:
queryFile = 'C:\Users\nroy0\Download\Thesis\dataset\input\query_train.txt'
ftrain = open(queryFile, 'w+')
ftrain.write('<parameters>\n')
for i in range(len(train)):
    ftrain.write('<query>\n')
    ftrain.write('<number>'+ str(train['QID'][i]) +'</number>\n')
    question_text = prepare_question_text(train['QID'][i], train['Question'][i])
    ftrain.write('<text>'+ question_text +'</text>\n')
    docid = train['DocumentID'][i]
    for item in docs[str(train['DocumentID'][i])].keys():
        ftrain.write('<workingSetDocno>'+str(train['QID'][i])+'-'+str(item)+ '</workingSetDocno>\n')
    
    ftrain.write('</query>\n')
ftrain.write('</parameters>')
ftrain.close()

In [6]:
words_file = open('words.txt', 'r')
all_words = words_file.read()
words_file.close()
all_words = all_words.split('\n')

In [18]:
prepare_question_text(train['QID'][1], train['Question'][1])


'assault bastille year revolution ultimately culminate capture execution louis xvi january 1793'

In [32]:
queryFile = 'query_train_mod.txt'
ftrain = open(queryFile, 'w+')
ftrain.write('<parameters>\n')
for i in range(len(train)):
    ftrain.write('<query>\n')
    ftrain.write('<number>'+ str(train['QID'][i]) +'</number>\n')
    question_text = prepare_question_text(train['QID'][i], train['Question'][i])
    ftrain.write('<text>'+ question_text +'</text>\n')
    docid = train['DocumentID'][i]
    for item in docs[str(train['DocumentID'][i])].keys():
        ftrain.write('<workingSetDocno>'+str(docid)+'-'+str(item)+ '</workingSetDocno>\n')
    
    ftrain.write('</query>\n')
ftrain.write('</parameters>')
ftrain.close()

In [36]:
ftest = open('C:/Users/nroy0/Downloads/Thesis/Lemur/dataset/input/query_test.txt', 'w+')
ftest.write('<parameters>\n')
for i in range(len(test)):
    if(i != 259):
        
        ftest.write('<query>\n')
        ftest.write('<number>'+ str(test['QID'][i]) +'</number>\n')
        question_text = prepare_question_text(test['QID'][i], test['Question'][i])
        ftest.write('<text>'+ question_text +'</text>\n')
        docid = test['DocumentID'][i]
        for item in docs[str(test['DocumentID'][i])].keys():
            ftest.write('<workingSetDocno>'+str(docid)+'-'+str(item)+ '</workingSetDocno>\n')

        ftest.write('</query>\n')
ftest.write('</parameters>')
ftest.close()

In [19]:
ftest = open('query_test_new.txt', 'w+')
ftest.write('<parameters>\n')
for i in range(len(test)):
    ftest.write('<query>\n')
    ftest.write('<number>'+ str(test['QID'][i]) +'</number>\n')
    question_text = prepare_question_text(test['QID'][i], test['Question'][i])
    ftest.write('<text>'+ question_text +'</text>\n')
    docid = test['DocumentID'][i]
    for item in docs[str(test['DocumentID'][i])].keys():
        ftest.write('<workingSetDocno>'+str(docid)+'-'+str(item)+ '</workingSetDocno>\n')
    
    ftest.write('</query>\n')
ftest.write('</parameters>')
ftest.close()

In [34]:
fdev = open('query_dev.txt', 'w+')
fdev.write('<parameters>\n')
for i in range(len(dev)):
    fdev.write('<query>\n')
    fdev.write('<number>'+ str(dev['QID'][i]) +'</number>\n')
    question_text = prepare_question_text(dev['QID'][i], dev['Question'][i])
    fdev.write('<text>'+ question_text +'</text>\n')
    docid = dev['DocumentID'][i]
    for item in docs[str(dev['DocumentID'][i])].keys():
        fdev.write('<workingSetDocno>'+str(docid)+'-'+str(item)+ '</workingSetDocno>\n')
    
    fdev.write('</query>\n')
fdev.write('</parameters>')
fdev.close()

## FINDING BEST MU FOR QUERY LIKELIHOOD 

In [13]:
for mu in [250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 5000]:
    startTime = time.time()
    queryFile = 'query_train.txt'
    outputFile = '1/QL/QL_['+str(mu)+'].indri'
    command_input= r'"C:\Users\nroy0\Downloads\Thesis\Lemur\bin\IndriRunQuery.exe" C:/Users/nroy0/Downloads/Thesis/Lemur/dataset/input/'+str(queryFile)+' -index=C:/Users/nroy0/Downloads/Thesis/Lemur/dataset/index/WPQA_mod -count=1000 -trecFormat=true -queryOffset=-1 -rule=method:dirichlet,mu:' +str(mu)+' -stemmer.name=krovetz > C:/Users/nroy0/Downloads/Thesis/trec_eval.9.0/baseline/'+str(outputFile) 
    args = shlex.split(command_input)
    p = subprocess.run(args, stderr=subprocess.STDOUT,shell=True)
    endTime = time.time()
    t_sec = round(endTime - startTime)
    (t_min, t_sec) = divmod(t_sec,60)
    print("Mu {} complete, time take {} min and {} sec".format(mu, t_min, t_sec ))

Mu 250 complete, time take 0 min and 0 sec
Mu 500 complete, time take 0 min and 17 sec
Mu 750 complete, time take 0 min and 17 sec
Mu 1000 complete, time take 0 min and 18 sec
Mu 1250 complete, time take 0 min and 18 sec
Mu 1500 complete, time take 0 min and 18 sec
Mu 1750 complete, time take 0 min and 18 sec
Mu 2000 complete, time take 0 min and 18 sec
Mu 2250 complete, time take 0 min and 18 sec
Mu 2500 complete, time take 0 min and 16 sec
Mu 2750 complete, time take 0 min and 17 sec
Mu 3000 complete, time take 0 min and 16 sec
Mu 5000 complete, time take 0 min and 18 sec


## FINDING BEST PARAMETERS FOR BM25

In [87]:
for k1 in [0, 0.4, 0.8, 1.2, 1.6, 2]:
    for b in [0, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1]:
        for k3 in [0, 1, 3, 5, 7, 9, 11 ]:
            startTime = time.time()
            queryFile = 'query_train.txt'
            outputFile = '1/BM25/BM_['+str(k1)+'_'+str(b)+'_'+str(k3)+'].indri'
            command_input= r'"C:\Users\nroy0\Downloads\Thesis\Lemur\bin\IndriRunQuery.exe" C:/Users/nroy0/Downloads/Thesis/Lemur/dataset/input/'+str(queryFile)+' -index=C:/Users/nroy0/Downloads/Thesis/Lemur/dataset/index/WPQA -count=1000 -trecFormat=true -queryOffset=-1 -baseline=bm25,k1:'+str(k1)+',b:'+str(b)+',k3:' +str(k3)+' -stemmer.name=krovetz > C:/Users/nroy0/Downloads/Thesis/trec_eval.9.0/baseline/'+str(outputFile) 
            args = shlex.split(command_input)
            p = subprocess.run(args, stderr=subprocess.STDOUT,shell=True)
            endTime = time.time()
            t_sec = round(endTime - startTime)
            (t_min, t_sec) = divmod(t_sec,60)
            print("k1:{}, b:{}, k3:{} complete, time taken {} min and {} sec".format(k1, b, k3, t_min, t_sec ))

k1:0, b:0, k3:0 complete, time taken 0 min and 30 sec
k1:0, b:0, k3:1 complete, time taken 0 min and 30 sec
k1:0, b:0, k3:3 complete, time taken 0 min and 31 sec
k1:0, b:0, k3:5 complete, time taken 0 min and 29 sec
k1:0, b:0, k3:7 complete, time taken 0 min and 30 sec
k1:0, b:0, k3:9 complete, time taken 0 min and 31 sec
k1:0, b:0, k3:11 complete, time taken 0 min and 31 sec
k1:0, b:0.15, k3:0 complete, time taken 0 min and 29 sec
k1:0, b:0.15, k3:1 complete, time taken 0 min and 30 sec
k1:0, b:0.15, k3:3 complete, time taken 0 min and 30 sec
k1:0, b:0.15, k3:5 complete, time taken 0 min and 32 sec
k1:0, b:0.15, k3:7 complete, time taken 0 min and 31 sec
k1:0, b:0.15, k3:9 complete, time taken 0 min and 32 sec
k1:0, b:0.15, k3:11 complete, time taken 0 min and 32 sec
k1:0, b:0.3, k3:0 complete, time taken 0 min and 31 sec
k1:0, b:0.3, k3:1 complete, time taken 0 min and 31 sec
k1:0, b:0.3, k3:3 complete, time taken 0 min and 31 sec
k1:0, b:0.3, k3:5 complete, time taken 0 min and 31 s

k1:0.8, b:0.6, k3:7 complete, time taken 0 min and 14 sec
k1:0.8, b:0.6, k3:9 complete, time taken 0 min and 14 sec
k1:0.8, b:0.6, k3:11 complete, time taken 0 min and 13 sec
k1:0.8, b:0.75, k3:0 complete, time taken 0 min and 14 sec
k1:0.8, b:0.75, k3:1 complete, time taken 0 min and 14 sec
k1:0.8, b:0.75, k3:3 complete, time taken 0 min and 13 sec
k1:0.8, b:0.75, k3:5 complete, time taken 0 min and 14 sec
k1:0.8, b:0.75, k3:7 complete, time taken 0 min and 14 sec
k1:0.8, b:0.75, k3:9 complete, time taken 0 min and 13 sec
k1:0.8, b:0.75, k3:11 complete, time taken 0 min and 14 sec
k1:0.8, b:0.9, k3:0 complete, time taken 0 min and 14 sec
k1:0.8, b:0.9, k3:1 complete, time taken 0 min and 14 sec
k1:0.8, b:0.9, k3:3 complete, time taken 0 min and 13 sec
k1:0.8, b:0.9, k3:5 complete, time taken 0 min and 13 sec
k1:0.8, b:0.9, k3:7 complete, time taken 0 min and 14 sec
k1:0.8, b:0.9, k3:9 complete, time taken 0 min and 14 sec
k1:0.8, b:0.9, k3:11 complete, time taken 0 min and 13 sec
k1:0

k1:2, b:0, k3:11 complete, time taken 0 min and 13 sec
k1:2, b:0.15, k3:0 complete, time taken 0 min and 14 sec
k1:2, b:0.15, k3:1 complete, time taken 0 min and 13 sec
k1:2, b:0.15, k3:3 complete, time taken 0 min and 13 sec
k1:2, b:0.15, k3:5 complete, time taken 0 min and 13 sec
k1:2, b:0.15, k3:7 complete, time taken 0 min and 13 sec
k1:2, b:0.15, k3:9 complete, time taken 0 min and 13 sec
k1:2, b:0.15, k3:11 complete, time taken 0 min and 13 sec
k1:2, b:0.3, k3:0 complete, time taken 0 min and 13 sec
k1:2, b:0.3, k3:1 complete, time taken 0 min and 13 sec
k1:2, b:0.3, k3:3 complete, time taken 0 min and 13 sec
k1:2, b:0.3, k3:5 complete, time taken 0 min and 13 sec
k1:2, b:0.3, k3:7 complete, time taken 0 min and 13 sec
k1:2, b:0.3, k3:9 complete, time taken 0 min and 13 sec
k1:2, b:0.3, k3:11 complete, time taken 0 min and 13 sec
k1:2, b:0.45, k3:0 complete, time taken 0 min and 14 sec
k1:2, b:0.45, k3:1 complete, time taken 0 min and 13 sec
k1:2, b:0.45, k3:3 complete, time take

In [78]:
p = Popen("C:/cygwin64/bin/bash.exe", stdin=PIPE, stdout=PIPE)
p.stdin.write(b'cd $HOME/cygdrive/c/Users/nroy0/Downloads/Thesis/trec_eval.9.0; > newfile5.txt;')
p.stdin.close()
out = p.stdout.read()
print (out)

b''


In [81]:
os.chdir(r"C:\Users\nroy0\Downloads\Thesis\Lemur\WikiPassageQA")
# cyg = "bash -c cd $Home/cygdrive/c/Users/nroy0/Downloads/Thesis/trec_eval.9.0; > newfile4.txt;"
# cmd = ["ls"]
# cmd = shlex.split(cyg)
# cmd = ['cd $Home/cygdrive/c/Users/nroy0/Downloads/Thesis/trec_eval.9.0; ./trec.sh; exit 123']
# ret=subprocess.call(cmd)
# pp = subprocess.Popen(cmd, stderr=subprocess.STDOUT,shell=True)
# find ./baseline/1 -name '*.indri' -exec sh -c './trec_eval -m all_trec baseline/qrel_train.txt ${0} > ${0}_output.csv' {} \;

In [82]:
os.getcwd()

'C:\\Users\\nroy0\\Downloads\\Thesis\\Lemur\\WikiPassageQA'

## READING THE TREC_EVAL RESULTS FILE INTO A DATAFRAME

In [5]:
path = r'C:\Users\nroy0\Downloads\Thesis\trec_eval.9.0\baseline\1\QL'
filenames = glob.glob(path + "/*.csv")
dfs = []
count = 0
for filename in filenames:
    match = re.search(r'\w\w\_\[\d+\]', filename)
    count = count + 1
#     print(count)
    colname = match.group()
    df = pd.read_table(filename,sep='\t', index_col=None, header=None, usecols=[2], skiprows =1, names =  [colname] , quoting=csv.QUOTE_NONE)
    dfs.append(df)
resultQL = pd.concat(dfs, axis=1, ignore_index = False)

In [98]:
path = r'C:\Users\nroy0\Downloads\Thesis\trec_eval.9.0\baseline\1\BM25'
filenames = glob.glob(path + "/*.csv")
dfs = []
count = 0
for filename in filenames:
#     print(filename)
    match = re.search(r'\w\w\_\[\d\.?\d*\_\d\.?\d*\_\d+\]', filename)
    count = count + 1
#     print(count)
    colname = match.group()
    df = pd.read_table(filename,sep='\t', index_col=None, header=None, usecols=[2], skiprows =1, names =  [colname] , quoting=csv.QUOTE_NONE)
    dfs.append(df)
resultBM = pd.concat(dfs, axis=1, ignore_index = False)

In [6]:
resultQL['index']=[ 'num_q',
'num_ret',
'num_rel',
'num_rel_ret',
'map',
'gm_map',
'Rprec',
'bpref',
'recip_rank',
'iprec_at_recall_0.00',
'iprec_at_recall_0.10',
'iprec_at_recall_0.20',
'iprec_at_recall_0.30',
'iprec_at_recall_0.40',
'iprec_at_recall_0.50',
'iprec_at_recall_0.60',
'iprec_at_recall_0.70',
'iprec_at_recall_0.80',
'iprec_at_recall_0.90',
'iprec_at_recall_1.00',
'P_5',
'P_10',
'P_15',
'P_20',
'P_30',
'P_100',
'P_200',
'P_500',
'P_1000',
'recall_5',
'recall_10',
'recall_15',
'recall_20',
'recall_30',
'recall_100',
'recall_200',
'recall_500',
'recall_1000',
'infAP',
'gm_bpref',
'Rprec_mult_0.20',
'Rprec_mult_0.40',
'Rprec_mult_0.60',
'Rprec_mult_0.80',
'Rprec_mult_1.00',
'Rprec_mult_1.20',
'Rprec_mult_1.40',
'Rprec_mult_1.60',
'Rprec_mult_1.80',
'Rprec_mult_2.00',
'utility',
'11pt_avg',
'binG',
'G',
'ndcg',
'ndcg_rel',
'Rndcg',
'ndcg_cut_5',
'ndcg_cut_10',
'ndcg_cut_15',
'ndcg_cut_20',
'ndcg_cut_30',
'ndcg_cut_100',
'ndcg_cut_200',
'ndcg_cut_500',
'ndcg_cut_1000',
'map_cut_5',
'map_cut_10',
'map_cut_15',
'map_cut_20',
'map_cut_30',
'map_cut_100',
'map_cut_200',
'map_cut_500',
'map_cut_1000',
'relative_P_5',
'relative_P_10',
'relative_P_15',
'relative_P_20',
'relative_P_30',
'relative_P_100',
'relative_P_200',
'relative_P_500',
'relative_P_1000',
'success_1',
'success_5',
'success_10',
'set_P',
'set_relative_P',
'set_recall',
'set_map',
'set_F',
'num_nonrel_judged_ret']
resultQL = resultQL.set_index('index')
resultQL

,QL_[1000],QL_[1250],QL_[1500],QL_[1750],QL_[2000],QL_[2250],QL_[2500],QL_[2750],QL_[3000],QL_[5000],QL_[500],QL_[750]
index,,,,,,,,,,,,
num_q,3330.0000,3330.0000,3330.0000,3330.0000,3330.0000,3330.0000,3330.0000,3330.0000,3330.0000,3330.0000,3330.0000,3330.0000
num_ret,194231.0000,194231.0000,194231.0000,194231.0000,194231.0000,194231.0000,194231.0000,194231.0000,194231.0000,194231.0000,194231.0000,194231.0000
num_rel,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000
num_rel_ret,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000,5556.0000
map,0.5189,0.5169,0.5143,0.5128,0.5116,0.5095,0.5078,0.5057,0.5042,0.4919,0.5207,0.5207
gm_map,0.3371,0.3349,0.3319,0.3296,0.3278,0.3257,0.3240,0.3219,0.3201,0.3075,0.3402,0.3397
Rprec,0.4186,0.4158,0.4131,0.4123,0.4115,0.4097,0.4073,0.4061,0.4044,0.3902,0.4200,0.4212
bpref,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
recip_rank,0.6076,0.6055,0.6018,0.5992,0.5974,0.5950,0.5931,0.5900,0.5885,0.5745,0.6110,0.6104


In [5]:
resultBM['index']=[ 'num_q',
'num_ret',
'num_rel',
'num_rel_ret',
'map',
'gm_map',
'Rprec',
'bpref',
'recip_rank',
'iprec_at_recall_0.00',
'iprec_at_recall_0.10',
'iprec_at_recall_0.20',
'iprec_at_recall_0.30',
'iprec_at_recall_0.40',
'iprec_at_recall_0.50',
'iprec_at_recall_0.60',
'iprec_at_recall_0.70',
'iprec_at_recall_0.80',
'iprec_at_recall_0.90',
'iprec_at_recall_1.00',
'P_5',
'P_10',
'P_15',
'P_20',
'P_30',
'P_100',
'P_200',
'P_500',
'P_1000',
'recall_5',
'recall_10',
'recall_15',
'recall_20',
'recall_30',
'recall_100',
'recall_200',
'recall_500',
'recall_1000',
'infAP',
'gm_bpref',
'Rprec_mult_0.20',
'Rprec_mult_0.40',
'Rprec_mult_0.60',
'Rprec_mult_0.80',
'Rprec_mult_1.00',
'Rprec_mult_1.20',
'Rprec_mult_1.40',
'Rprec_mult_1.60',
'Rprec_mult_1.80',
'Rprec_mult_2.00',
'utility',
'11pt_avg',
'binG',
'G',
'ndcg',
'ndcg_rel',
'Rndcg',
'ndcg_cut_5',
'ndcg_cut_10',
'ndcg_cut_15',
'ndcg_cut_20',
'ndcg_cut_30',
'ndcg_cut_100',
'ndcg_cut_200',
'ndcg_cut_500',
'ndcg_cut_1000',
'map_cut_5',
'map_cut_10',
'map_cut_15',
'map_cut_20',
'map_cut_30',
'map_cut_100',
'map_cut_200',
'map_cut_500',
'map_cut_1000',
'relative_P_5',
'relative_P_10',
'relative_P_15',
'relative_P_20',
'relative_P_30',
'relative_P_100',
'relative_P_200',
'relative_P_500',
'relative_P_1000',
'success_1',
'success_5',
'success_10',
'set_P',
'set_relative_P',
'set_recall',
'set_map',
'set_F',
'num_nonrel_judged_ret']
resultBM = resultBM.set_index('index')
resultBM

NameError: name 'resultBM' is not defined

In [95]:
import re
match = re.search(r'\w\w\_\[\d\.?\d*\_\d\.?\d*\_\d+\]', 'C:\\Users\\nroy0\\Downloads\\Thesis\\trec_eval.9.0\\baseline\\1\\QL_[1.2_1_13].indri_output.csv' )
match.group()

'QL_[1.2_1_13]'

In [1]:
resultQL.loc['ndcg_cut_10'].idxmax(axis=1)

NameError: name 'resultQL' is not defined

In [126]:
resultQL['QL_[500]']

index
num_q                      3330.0000
num_ret                  194231.0000
num_rel                    5556.0000
num_rel_ret                5556.0000
map                           0.5207
gm_map                        0.3402
Rprec                         0.4198
bpref                         1.0000
recip_rank                    0.6108
iprec_at_recall_0.00          0.6178
iprec_at_recall_0.10          0.6178
iprec_at_recall_0.20          0.6177
iprec_at_recall_0.30          0.6156
iprec_at_recall_0.40          0.5995
iprec_at_recall_0.50          0.5990
iprec_at_recall_0.60          0.4457
iprec_at_recall_0.70          0.4456
iprec_at_recall_0.80          0.4325
iprec_at_recall_0.90          0.4324
iprec_at_recall_1.00          0.4324
P_5                           0.1880
P_10                          0.1139
P_15                          0.0833
P_20                          0.0667
P_30                          0.0482
P_100                         0.0166
P_200                         0.

## TEST SET

In [29]:
def testTrec(mu):
    startTime = time.time()
    queryFile = 'query_test.txt'
    outputFile = '1/QL/Test/QL_new_['+str(mu)+'].indri'
    command_input= r'"C:\Users\nroy0\Downloads\Thesis\Lemur\bin\IndriRunQuery.exe" C:/Users/nroy0/Downloads/Thesis/Lemur/dataset/input/'+str(queryFile)+' -index=C:/Users/nroy0/Downloads/Thesis/Lemur/dataset/index/WPQA_mod -count=1000 -trecFormat=true -queryOffset=-1 -rule=method:dirichlet,mu:' +str(mu)+' -stemmer.name=krovetz > C:/Users/nroy0/Downloads/Thesis/trec_eval.9.0/baseline/'+str(outputFile) 
    args = shlex.split(command_input)
    p = subprocess.run(args, stderr=subprocess.STDOUT,shell=True)
    endTime = time.time()
    t_sec = round(endTime - startTime)
    (t_min, t_sec) = divmod(t_sec,60)
    print("Mu {} complete, time take {} min and {} sec".format(mu, t_min, t_sec ))

In [37]:
testTrec(750)

Mu 750 complete, time take 0 min and 1 sec


In [10]:
path = r'C:\Users\nroy0\Downloads\Thesis\trec_eval.9.0\baseline\1\QL\Test'
filenames = glob.glob(path + "/*.csv")
dfs = []
count = 0
for filename in filenames:
    match = re.search(r'\w\w\_\[\d+\]', filename)
    count = count + 1
#     print(count)
    colname = match.group()
    df = pd.read_table(filename,sep='\t', index_col=None, header=None, usecols=[2], skiprows =1, names =  [colname] , quoting=csv.QUOTE_NONE)
    dfs.append(df)
resultQLTest = pd.concat(dfs, axis=1, ignore_index = False)

resultQLTest['index']=[ 'num_q',
'num_ret',
'num_rel',
'num_rel_ret',
'map',
'gm_map',
'Rprec',
'bpref',
'recip_rank',
'iprec_at_recall_0.00',
'iprec_at_recall_0.10',
'iprec_at_recall_0.20',
'iprec_at_recall_0.30',
'iprec_at_recall_0.40',
'iprec_at_recall_0.50',
'iprec_at_recall_0.60',
'iprec_at_recall_0.70',
'iprec_at_recall_0.80',
'iprec_at_recall_0.90',
'iprec_at_recall_1.00',
'P_5',
'P_10',
'P_15',
'P_20',
'P_30',
'P_100',
'P_200',
'P_500',
'P_1000',
'recall_5',
'recall_10',
'recall_15',
'recall_20',
'recall_30',
'recall_100',
'recall_200',
'recall_500',
'recall_1000',
'infAP',
'gm_bpref',
'Rprec_mult_0.20',
'Rprec_mult_0.40',
'Rprec_mult_0.60',
'Rprec_mult_0.80',
'Rprec_mult_1.00',
'Rprec_mult_1.20',
'Rprec_mult_1.40',
'Rprec_mult_1.60',
'Rprec_mult_1.80',
'Rprec_mult_2.00',
'utility',
'11pt_avg',
'binG',
'G',
'ndcg',
'ndcg_rel',
'Rndcg',
'ndcg_cut_5',
'ndcg_cut_10',
'ndcg_cut_15',
'ndcg_cut_20',
'ndcg_cut_30',
'ndcg_cut_100',
'ndcg_cut_200',
'ndcg_cut_500',
'ndcg_cut_1000',
'map_cut_5',
'map_cut_10',
'map_cut_15',
'map_cut_20',
'map_cut_30',
'map_cut_100',
'map_cut_200',
'map_cut_500',
'map_cut_1000',
'relative_P_5',
'relative_P_10',
'relative_P_15',
'relative_P_20',
'relative_P_30',
'relative_P_100',
'relative_P_200',
'relative_P_500',
'relative_P_1000',
'success_1',
'success_5',
'success_10',
'set_P',
'set_relative_P',
'set_recall',
'set_map',
'set_F',
'num_nonrel_judged_ret']
resultQLTest = resultQLTest.set_index('index')
resultQLTest

,QL_[500],QL_[750]
index,,
num_q,416.0000,416.0000
num_ret,23981.0000,23981.0000
num_rel,700.0000,700.0000
num_rel_ret,700.0000,700.0000
map,0.5283,0.5223
gm_map,0.3431,0.3364
Rprec,0.4302,0.4246
bpref,1.0000,1.0000
recip_rank,0.6189,0.6138


In [2]:
resultQLTest.loc[['map', 'recip_rank','recall_10', 'P_10', 'ndcg', 'ndcg_cut_10']]

NameError: name 'resultQLTest' is not defined

## Sanity Check

In [34]:
queryFile = 'queryTestSanity1.txt'
ftrain = open(queryFile, 'w+')
ftrain.write('<parameters>\n')
for i in range(len(test)):
    ftrain.write('<query>\n')
#     print(i)
    ftrain.write('<number>'+ str(test['QID'][i]) +'</number>\n')
    question_text = prepare_question_text(test['QID'][i], test['Question'][i])
    randomWord1 = random.choice(all_words)
    randomWord2 = random.choice(all_words)
    randomWord3 = random.choice(all_words)
    randomWord4 = random.choice(all_words)
    ftrain.write('<text>'+randomWord1 +' '+randomWord2+' '+randomWord3+' '+randomWord4+ '</text>\n')
    docid = test['DocumentID'][i]
    for item in docs[str(test['DocumentID'][i])].keys():
        ftrain.write('<workingSetDocno>'+str(docid)+'-'+str(item)+ '</workingSetDocno>\n')
    
    ftrain.write('</query>\n')
ftrain.write('</parameters>')
ftrain.close()

In [37]:
def testTrec(mu):
    startTime = time.time()
    queryFile = 'queryTestSanity1.txt'
    outputFile = '1/QL/Test/SanQL2_['+str(mu)+'].indri'
    command_input= r'"C:\Users\nroy0\Downloads\Thesis\Lemur\bin\IndriRunQuery.exe" C:/Users/nroy0/Downloads/Thesis/Lemur/dataset/input/'+str(queryFile)+' -index=C:/Users/nroy0/Downloads/Thesis/Lemur/dataset/index/WPQA_mod -count=1000 -trecFormat=true -queryOffset=-1 -rule=method:dirichlet,mu:' +str(mu)+' -stemmer.name=krovetz > C:/Users/nroy0/Downloads/Thesis/trec_eval.9.0/baseline/'+str(outputFile) 
    args = shlex.split(command_input)
    p = subprocess.run(args, stderr=subprocess.STDOUT,shell=True)
    endTime = time.time()
    t_sec = round(endTime - startTime)
    (t_min, t_sec) = divmod(t_sec,60)
    print("Mu {} complete, time take {} min and {} sec".format(mu, t_min, t_sec ))

In [23]:
testTrec(500)

Mu 500 complete, time take 0 min and 1 sec


In [ ]:
def prepare_question_text(q_id, question_text):
    question_text = question_text.replace('?','') #remove ?
    question_text = question_text.replace("'",'')
    question_text = question_text.replace('"','')
    question_text = question_text.replace('-',' ')
    question_text = question_text.replace('(','')
    question_text = question_text.replace(')','')
    question_text = question_text.replace(',','')
    question_text = question_text.replace('.','')
    question_text = question_text.replace('&',' and ')
    question_text = question_text.replace(':','')
    question_text = question_text.replace('>','')#error in dataset

    if "[" in question_text:
        #print q_id,question_text
        if q_id == "3340": #remove contents
            question_text = re.sub(r'\[[^\(]*?\]', r'', question_text)
        else: #keep contents
            question_text = re.sub(r'\[(?:[^\]|]*\|)?([^\]|]*)\]', r'\1', question_text)
    if "/" in question_text:
        #print q_id,question_text
        if q_id == "104" or q_id == "857":
            question_text = question_text.replace('/','')
        else:
            question_text = question_text.replace('/',' or ')

    return question_text